In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install -U transformers accelerate datasets sentencepiece

from pathlib import Path

ruta_dataset = "/content/drive/MyDrive/poemas/resultados.txt"

raw_text = Path(ruta_dataset).read_text(encoding="utf-8")

print("Primeros 500 caracteres del archivo:")
print(raw_text[:500])
poems = [p.strip() for p in raw_text.split("---") if p.strip()]

dataset_list = []
for poem in poems:
    entry = {
        "text":
f"""Escribe un poema usando la figura retórica "anáfora".
Poema:
{poem}
"""
    }
    dataset_list.append(entry)

len(dataset_list)
from datasets import Dataset
hf_dataset = Dataset.from_list(dataset_list)
hf_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

MODEL_NAME = "DeepESP/gpt2-spanish"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)
def tokenize(batch):
    tokens = tokenizer(
        batch["text"],
        truncation=True,
        max_length=256
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_dataset = hf_dataset.map(tokenize, batched=True, remove_columns=["text"])
tokenized_dataset

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="modelo_anafora",
    overwrite_output_dir=True,

    num_train_epochs=5,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,

    learning_rate=3e-5,
    weight_decay=0.1,
    warmup_ratio=0.05,

    lr_scheduler_type="cosine",

    fp16=True,
    logging_steps=20,
    save_steps=300,
    save_total_limit=2,

    report_to="none",
    optim="adamw_torch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

trainer.train()

trainer.save_model("modelo_anafora")
tokenizer.save_pretrained("modelo_anafora")

!cp -r modelo_anafora /content/drive/MyDrive/poemas/modelo_anafora/


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Primeros 500 caracteres del archivo:
 Río corre su curso tranquilo,
Río esconde secretos antiguos,
Río llena las costas secas,
Río nunca se detiene.

---

 Las flores blanquean el camino,
Las flores cantan sus himnos,
Las flores abrazan el sol,
Las flores mueren en su gloria.

---

 Nieve cubre montañas altas,
Nieve se esconde en zonas ocultas,
Nieve flota sobre ramas viejas,
Nieve mira hacia las estrellas blancas.

---

 Montaña alta y solitaria,
Montaña resplandeciente en la luz del sol,
Montaña sagrada y misteriosa,
Montaña que 


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
20,2.791400
40,1.543400
60,1.346400
80,1.243200
